In [20]:
using CSV, DataFrames, Statistics, Random, Plots, JuMP, Gurobi

In [21]:
using ScikitLearn
@sk_import linear_model: LinearRegression
@sk_import linear_model: Lasso
@sk_import model_selection: train_test_split

┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore ~/.julia/packages/ScikitLearn/sqLdT/src/Skcore.jl:259


PyObject <function train_test_split at 0x2b902dea0>

In [31]:
function opt_split(X, y, n, p, k, lambda, weight = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)
    p = p + 1

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, theta)
    @variable(model, u[1:n] >= 0)
    @variable(model, beta[1:p])
    @variable(model, w[1:p])

    @objective(model, Min, k * theta + sum(u) + lambda * sum(w))

    for i in 1:p
        @constraint(model, w[i] >= beta[i])
        @constraint(model, w[i] >= -beta[i])
    end 

    for i in 1:n
        if weight == nothing
            @constraint(model, theta + u[i] >= y[i] - sum(X[i, :].*beta))
            @constraint(model, theta + u[i] >= -(y[i] - sum(X[i, :].*beta)))
        else 
            @constraint(model, theta + u[i] >=  weight[i] * (y[i] - sum(X[i, :].*beta)))
            @constraint(model, theta + u[i] >= - weight[i] * (y[i] - sum(X[i, :].*beta)))
        end 
    end
    
    optimize!(model)

    return value(theta), value.(u), value.(beta), value.(w)
end

opt_split (generic function with 2 methods)

In [32]:
function split_train_val(X, y, beta_star, train_fraction = 0.8, weights = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)

    residuals = y - X * beta_star

    if weights == nothing
        sorted_indices = sortperm(abs.(residuals), rev=true)
    else 
        residuals_weights = [residuals[i] * weights[i] for i in 1:length(weights)]
        sorted_indices = sortperm(abs.(residuals_weights), rev=true)
    end

    num_train_points = round(Int, train_fraction * length(sorted_indices))

    train_indices = sorted_indices[1:num_train_points]

    val_indices = setdiff(1:length(y), train_indices)

    X_train = X[train_indices, 2:end]
    y_train = y[train_indices, :]

    X_val = X[val_indices, 2:end]
    y_val = y[val_indices, :]

    return X_train, y_train, X_val, y_val
end

split_train_val (generic function with 3 methods)

In [68]:
function fit_evaluate_model(X_train, X_val, X_test, y_train, y_val, y_test, lambdas)
    current_lambda = Inf
    current_val_mse = Inf
    current_test_mse = Inf
    
    for lambda in lambdas

        model = Lasso(lambda=lambda)
        fit!(model, X_train, y_train)
        
        y_pred_val = predict(model, X_val)
        val_mse_i = mse(y_val, y_pred_val)
        
        if current_val_mse >= val_mse_i
            current_lambda = lambda
            current_val_mse = val_mse_i
            current_test_mse = mse(y_test, predict(model, X_test))
        end
    end

    # println("best lambda: ", current_lambda)
    # println("val score: ", current_val_mse)
    # println("test score: ", current_test_mse)

    return current_val_mse, current_test_mse
end

fit_evaluate_model (generic function with 1 method)

In [34]:
function mse(y_true, y_pred)
    mse = mean((y_true .- y_pred).^2)
    return mse
end

mse (generic function with 1 method)

### Abalone dataset

In [40]:
# Load the data
df = CSV.read("abalone_original.csv", DataFrame)
selected_columns = setdiff(names(df), ["rings", "sex"])

X = df[:,selected_columns]
y = CSV.read("abalone_original.csv", DataFrame)[!,"rings"];

(train_X, train_y), (test_X, test_y) =
    IAI.split_data(:regression, X, y, train_proportion=0.9, seed = 1);

In [41]:
#get optimised split 
n = nrow(train_X)
p = ncol(train_X)
train_fraction = 0.7
k = n * train_fraction
lambda = 0.001;

In [42]:
#get optimised split
_, _, betas_star, _ = opt_split(Array(train_X), Array(train_y), n, p, k, lambda)
X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(Array(train_X), Array(train_y), betas_star);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [48]:
#get non optimised split 
(X_train_rand, y_train_rand), (X_val_rand, y_val_rand) =
    IAI.split_data(:regression, train_X, train_y, train_proportion=train_fraction);

In [49]:
lambdas = [0.00001, 0.0001, 0.001, 0.01, 0.1]
fit_evaluate_model(X_train_opt, X_val_opt, Matrix(test_X), y_train_opt, y_val_opt, Array(test_y), lambdas)

best lambda: 1.0e-5
val score: 0.2795205385508074
test score: 4.907803006407442


/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.972e+02, tolerance: 3.554e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.237e+00, tolerance: 3.554e+00
  model = cd_fast.enet_coordinate_descent(


(0.2795205385508074, 4.907803006407442)

In [50]:
fit_evaluate_model(Matrix(X_train_rand), Matrix(X_val_rand), Matrix(test_X), Array(y_train_rand), y_val_rand, Array(test_y), lambdas)

best lambda: 0.1
val score: 4.5929545220999195
test score: 4.954459285312086


/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.813e+03, tolerance: 2.823e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.639e+02, tolerance: 2.823e+00
  model = cd_fast.enet_coordinate_descent(


(4.5929545220999195, 4.954459285312086)

## Do it 100 times

In [61]:
function repeat_stable_reg(X_full, y_full, num_repeats = 100)
    opt_mse_val_score = []
    opt_mse_test_score = []
    rand_mse_test_score = []
    rand_mse_val_score = []
    lambdas = [0.000001,0.00001, 0.0001, 0.001, 0.01, 0.1]

    lambda_4_opt = 0.001;
    train_fraction = 0.7 #spliting the train into train and val  

    for i in 1:num_repeats
    
        (train_X, train_y), (test_X, test_y) =
        IAI.split_data(:regression, X_full, y_full, train_proportion=0.9, seed = i);
        
        n = nrow(train_X)
        p = ncol(train_X)
        k = n * train_fraction

        #get opt split
        _, _, betas_star, _ = opt_split(Array(train_X), Array(train_y), n, p, k, lambda_4_opt)
        X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(Array(train_X), Array(train_y), betas_star);

        #get random split 
        (X_train_rand, y_train_rand), (X_val_rand, y_val_rand) = 
            IAI.split_data(:regression, train_X, train_y, train_proportion=train_fraction);

        opt_val_mse, opt_test_mse = fit_evaluate_model(X_train_opt, X_val_opt, Matrix(test_X), y_train_opt, y_val_opt, Array(test_y), lambdas)
        rand_val_mse, rand_test_mse = fit_evaluate_model(Matrix(X_train_rand), Matrix(X_val_rand), Matrix(test_X), Array(y_train_rand), y_val_rand, Array(test_y), lambdas)
        
        push!(opt_mse_val_score, opt_val_mse)
        push!(opt_mse_test_score, opt_test_mse)

        push!(rand_mse_val_score, rand_val_mse)
        push!(rand_mse_test_score, rand_test_mse)
            
    end

    return opt_mse_val_score, opt_mse_test_score, rand_mse_val_score, rand_mse_test_score

end

repeat_stable_reg (generic function with 2 methods)

In [62]:
function calculate_percentage_improvement(opt_scores, rand_scores)
    if length(opt_scores) != length(rand_scores)
        error("Old scores and new scores must have the same length.")
    end
    
    percentage_improvements = Float64[]

    for i in 1:length(opt_scores)
        opt_score = opt_scores[i]
        rand_score = rand_scores[i]

        percentage_improvement = ((rand_score - opt_score) / rand_score) * 100
        push!(percentage_improvements, percentage_improvement)
    end

    return mean(percentage_improvements)
end

calculate_percentage_improvement (generic function with 1 method)

In [63]:
opt_mse_val_score, opt_mse_test_score, rand_mse_val_score, rand_mse_test_score = repeat_stable_reg(X, y)

/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.319e+03, tolerance: 3.554e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.972e+02, tolerance: 3.554e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
index: 10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 

/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.839e+03, tolerance: 3.546e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.060e+02, tolerance: 3.546e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

20


/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.501e+03, tolerance: 3.557e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.027e+02, tolerance: 3.557e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

...

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
index: 90
Set parameter Username
Academic license - for non-commercial use only - expires 

/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.278e+03, tolerance: 3.564e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.854e+02, tolerance: 3.564e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

100
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.849e+03, tolerance: 3.464e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.243e+02, tolerance: 3.464e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

(Any[0.27952046954890863, 0.2724111991278995, 0.2902380191697328, 0.2658304092196116, 0.2636700314759561, 0.2821210286119788, 0.27313887977996804, 0.2717887399502597, 0.2766238345608332, 0.27678413620483217  …  0.27286538411546807, 0.26013938968039485, 0.2854739669651963, 0.2696410008059866, 0.3003020682694555, 0.29188386417684575, 0.28070922456339403, 0.2955705536736079, 0.26146711605026784, 0.28038311138537536], Any[4.907801596463567, 5.449067047589291, 5.4776253212583965, 5.422588055628983, 4.926781440628955, 5.329784023718956, 4.819808270432159, 4.6107365693184805, 6.064340495424297, 4.890343886107385  …  5.653041106812362, 5.229342564295244, 5.186741924142661, 5.199110121739272, 5.091448730115362, 4.480732092557713, 5.957212781192549, 4.816144548596554, 4.263545538070436, 5.768310947053256], Any[5.070284246585678, 4.713269879168693, 5.096194406872005, 4.652687809287293, 4.762378386889108, 5.274123634572568, 5.270833607984219, 5.553274467206096, 5.132449398058274, 4.760183724686175

In [66]:
meam_percentage_diff = calculate_percentage_improvement(opt_mse_test_score, rand_mse_test_score)

-0.08152634902540529